In [ ]:
# Note: restart runtime after this import before running the augmentations
!pip install -U augly[video]
!sudo apt-get install python3-magic

In [ ]:
# Installing ffmpeg is only needed for the video module of augly
!sudo add-apt-repository ppa:jonathonf/ffmpeg-4
!apt install ffmpeg

In [ ]:
from IPython.display import display, HTML
from base64 import b64encode

def display_video(path):
  mp4 = open(path, "rb").read()
  data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
  display(
    HTML(
      """
          <video width=400 controls>
                <source src="%s" type="video/mp4">
          </video>
      """ % data_url
    )
  )

In [ ]:
import os
import augly.utils as utils
import augly.video as vidaugs

# Get input video, trim to first 3 seconds
input_video = os.path.join(
    utils.TEST_URI, "video", "inputs", "input_1.mp4"
)
input_vid_path = "/tmp/in_video.mp4"
out_vid_path = "/tmp/aug_video.mp4"

# We can use the AugLy trim augmentation, and save the trimmed video
vidaugs.trim(input_video, output_path=input_vid_path, start=0, end=3)
display_video(input_vid_path)

In [ ]:
# Now we can apply various augmentations to the trimmed video!
vidaugs.overlay_text(input_vid_path, out_vid_path)
display_video(out_vid_path)

In [ ]:
"""
You can optionally pass in a metadata list, to which metadata about the
augmentation will be appended, including kwargs, input & output dimensions,
the matching segments in the input & output videos (useful in case of temporal
editing), and intensity (defined based on the kwargs for each augmentation).
"""
meta = []
vidaugs.loop(
    input_vid_path,
    out_vid_path,
    num_loops=1,
    metadata=meta,
)
display_video(out_vid_path)
meta

In [ ]:
# For all the augmentations, we have class-based definitions as well as
# functional
meta = []
aug = vidaugs.InsertInBackground()
aug(input_vid_path, out_vid_path, metadata=meta)
display_video(out_vid_path)
meta

In [ ]:
"""
For some augmentations, we also provide versions that will randomly sample
from a set of parameters (e.g. for OverlayEmoji, RandomEmojiOverlay samples
an emoji from Twitter's Twemoji set which we provide in the augly package).
The metadata will contain the actual sampled param values.
"""
meta = []
aug = vidaugs.RandomEmojiOverlay()
aug(input_vid_path, out_vid_path, metadata=meta)
display_video(out_vid_path)
meta

In [ ]:
# You can also compose several transformations together
aug = vidaugs.Compose(
    [
        vidaugs.AddNoise(),
        vidaugs.Blur(sigma=5.0),
        vidaugs.OverlayDots(),
    ]
)
aug(input_vid_path, out_vid_path)
display_video(out_vid_path)